In [102]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
from ase.io import read

In [103]:
molpro_folder='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Benzene'

files=[]
for (dirpath, dirnames, filenames) in os.walk(molpro_folder):
    for filename in filenames:
        files.append(f'{dirpath}/{filename}')

files=list(filter(lambda x:'ase.xyz' in x, files))

data_molpro=pd.DataFrame({
    'Calculation type': 'Ab initio',
    'Method': '',
    'Distance': [0]*len(files),
    'Energy': 0
})

In [104]:
for i, row in data_molpro.iterrows():
    data_molpro.loc[i, 'Method'] = files[i].split('/')[-4]
    data_molpro.loc[i, 'Distance'] = files[i].split('/')[-2].split('_')[-1]

    atoms=read(files[i],format='extxyz')
    data_molpro.loc[i, 'Energy'] = atoms.get_total_energy()

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_90891/1374398513.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.90' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_90891/1374398513.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-12627.129353934697' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [105]:
data_mace=pd.read_pickle('/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Benzene/MACE/results.pkl')
data_mace['Method']='MACE'+' '+data_mace['Model']

df=pd.concat([data_molpro,data_mace], ignore_index=True)

In [106]:
# calculate interaction energy (total energy minus monomers)
df['Interaction energy [eV]']=0

for method in df['Method'].unique():
    subset=df.query(f'Method == "{method}"')
    df.loc[df['Method']==method, 'Interaction energy [eV]'] = subset['Energy'] - float(subset.loc[subset['Distance']=='1', 'Energy']) - float(subset.loc[subset['Distance']=='2', 'Energy'])

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_90891/4006019392.py:6: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_90891/4006019392.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 3.63122542e-02 -1.11152012e-01  6.31358283e+03 -8.32900603e-02
 -6.53206351e-02 -2.93166973e-02 -1.25251575e-01 -4.07935387e-03
  6.31358284e+03 -1.22984097e-01]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_90891/4006019392.py:6: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_90891/4006019392.py:6:

In [107]:
df=df[df['Distance'].str.len()>=2] # drop full integer values (Distance 2 & 1 represent individual benzene and not dimer)
df['Distance']=df['Distance'].astype(float)
df=df.sort_values('Distance').reset_index(drop=True)

In [108]:
plot=px.line(df, x='Distance', y='Interaction energy [eV]', color='Method', markers=True, template='ggplot2',
             title='Benzene dimer: interaction energy in relation to distance (multiplier of equilibrium distance determined by CCSD(T) calculation). Model ANI-1ccx: MACE model trained on ANI-1ccx dataset (organic molecules CCSD(T) level). Model s66x8: MACE model trained on s66x8 dataset (organic dimer molecules CCSD(T) level). Model ANI-1ccx+s66x8: trained on both.')

In [109]:
plot.show()

In [110]:
plot.write_html('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Graphs/benzeneDimer_Distance.html')

In [111]:
# rename columns
df.loc[df['Method']=='DF-KS_PBE-D3_BJ', 'Method']='DFT PBE-D3BJ'
df.loc[df['Method']=='DF-KS_PBE', 'Method']='DFT PBE'
df.loc[df['Method']=='DF-KS_LDA', 'Method']='DFT LDA'
df.loc[df['Method']=='DF-PNO-LCCSD_T', 'Method']='LCCSD(T)'

In [112]:
# add distinctive colours

df['Colour']=''

for i, name in enumerate(df['Method'].unique()):
    if 'DFT PBE-D3BJ' in name:
        df.loc[(df['Method']==name),['Colour']]='red'
    elif 'LDA' in name:
        df.loc[(df['Method']==name),['Colour']]='turquoise'
    elif 'DFT PBE'==name:
        df.loc[(df['Method']==name),['Colour']]='grey'
    elif 'MACE ANI-1ccx'==name:
        df.loc[(df['Method']==name),['Colour']]='green'
    elif 'MACE ANI-1ccx+s66x8'==name:
            df.loc[(df['Method']==name),['Colour']]='orange'
    elif 'MACE s66x8'==name:
            df.loc[(df['Method']==name),['Colour']]='purple'
    elif 'LCCSD(T)' in name:
            df.loc[(df['Method']==name),['Colour']]='blue'
    elif 'LCCSD(T)' in name:
            df.loc[(df['Method']==name),['Colour']]='black'

In [113]:
# filter data and produce plot for use in thesis
df_fp = df.query("Method != 'DF-KS_PBE-D4'") # drop D4 data
df_fp = df_fp.query('not `Method`.str.contains("MACE")') 

In [114]:
df.loc[df['Method']=='DF-KS_PBE-D3_BJ', 'Method']

Series([], Name: Method, dtype: object)

In [115]:
plot=px.line(df_fp, x='Distance', y='Interaction energy [eV]', color='Method', markers=True, template='ggplot2')
plot.update_layout(font_family="Serif",font_size=18,
                   xaxis=dict(
                       tick0=0.9,
                       range=[0.9,2]
                   ),
                   yaxis=dict(
                       range=[-0.2,0.3]
                   ),
                   width=1000,
                   height=350)

In [116]:
plot.write_image('/Users/dominicwelti/Library/CloudStorage/Dropbox/Apps/Overleaf/Master Thesis - MLIP evaluation/graphs/results/benzeneDimer_ab_initio.pdf',format='pdf',scale=5)

In [117]:
df

,Calculation type,Method,Distance,Energy,Model,c Parameter,Interaction energy [eV],Colour
0,Ab initio,DFT PBE-D3BJ,0.9,-12627.129354,NaN,NaN,0.036312,red
1,Ab initio,DFT PBE,0.9,-12626.732479,NaN,NaN,0.267649,grey
2,Ab initio,DFT LDA,0.9,-12527.151069,NaN,NaN,-0.069390,turquoise
3,MACE,MACE ANI-1ccx+s66x8,0.9,-12616.290039,ANI-1ccx+s66x8,0.0,0.006836,orange
4,MACE,MACE s66x8,0.9,-12616.311523,s66x8,0.0,-0.007812,purple
...,...,...,...,...,...,...,...,...
59,MACE,MACE ANI-1ccx+s66x8,2.0,-12616.296875,ANI-1ccx+s66x8,0.0,0.000000,orange
60,MACE,MACE s66x8,2.0,-12616.303711,s66x8,0.0,0.000000,purple
61,Ab initio,DF-KS_PBE-D4,2.0,-12627.589096,NaN,NaN,-0.001668,
62,Ab initio,DFT PBE,2.0,-12626.998798,NaN,NaN,0.001330,grey


In [118]:
df_mace=df.query('`Method`.str.contains("MACE") or `Method`.str.contains("CCSD") or `Method`.str.contains("DFT PBE-D3BJ")')

In [119]:
plot=px.line(df_mace, x='Distance', y='Interaction energy [eV]', color='Method', markers=True, template='ggplot2')
plot.update_layout(font_family="Serif",font_size=18,
                   xaxis=dict(
                       tick0=0.9,
                       range=[0.9,2]
                   ),
                   yaxis=dict(
                       range=[-0.2,0.3]
                   ),
                   width=1000,
                   height=350,
                   )

In [120]:
plot.write_image('/Users/dominicwelti/Library/CloudStorage/Dropbox/Apps/Overleaf/Master Thesis - MLIP evaluation/graphs/results/benzeneDimer_mace.pdf',format='pdf',scale=5)

In [137]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# combined plots 
fig_c=make_subplots(rows=2,cols=1,
                    shared_xaxes=True,shared_yaxes=True,
                    y_title='Interaction energy [eV]',
                    subplot_titles=('(a)',  '(b)'))
subs=[df_fp,df_mace]
i=0

for data in subs:
    i+=1
    for method in data['Method'].unique():
        subset=data.query('Method==@method')
        print(i,method)
        fig_c.append_trace(go.Scatter(
            x=subset['Distance'],
            y=subset['Interaction energy [eV]'],
            #color=subset('Method==@method')['Colour'][0],
            name=method,
            line={'color':subset['Colour'].unique()[0]},
        ), row=i,col=1
    )
        

fig_c.update_layout(#yaxis2={'title':'Relative energy'},
                xaxis2={'title':'Equilibrium distance factor','range':[0.9,2]},
                grid_yaxes=['y1'],
                legend_title_text='Method',
                font_family="Serif",font_size=20,
                template='ggplot2',
                width=1000,
                height=600) #,'position':0


#remove duplicate labels
names=set()
fig_c.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))

for i,an in enumerate(fig_c.layout.annotations):
    fig_c.layout.annotations[i]["font"] = {'size': 23,'family':"Serif"}
#fig_c.layout.annotations[0]["font"] = {'size': 22}
fig_c.layout.annotations[2]["xshift"] = -50

1 DFT PBE-D3BJ
1 DFT PBE
1 DFT LDA
1 LCCSD(T)
2 DFT PBE-D3BJ
2 MACE ANI-1ccx+s66x8
2 MACE s66x8
2 MACE ANI-1ccx
2 LCCSD(T)


In [138]:
fig_c.show()

In [139]:
fig_c.write_image('/Users/dominicwelti/Library/CloudStorage/Dropbox/Apps/Overleaf/Master Thesis - MLIP evaluation/graphs/results/benzeneDimer_comb.pdf',format='pdf',scale=5)